In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Importing the necessary libraries for Colpali

In [2]:
!pip install -q colpali_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.1 MB/s eta 0:00:00:00:0100:01


In [3]:
from colpali_engine.models import ColPali
from colpali_engine.models.paligemma.colpali.processing_colpali import ColPaliProcessor
from colpali_engine.utils.processing_utils import BaseVisualRetrieverProcessor
from colpali_engine.utils.torch_utils import ListDataset,get_torch_device
from torch.utils.data import DataLoader

import torch
from typing import List,cast
from tqdm import tqdm
from PIL import Image

# import spaces
import warnings
warnings.filterwarnings('ignore')

2025-05-18 10:55:35.566873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747565735.744587      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747565735.797063      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 2. Initialting the model

In [4]:
model_name='vidore/colpali-v1.2'
device=get_torch_device('cuda')
print(device)

cuda


In [5]:
model=ColPali.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=device
).eval()

processor=cast(ColPaliProcessor,ColPaliProcessor.from_pretrained(model_name))

adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

# 3. Converting all the PDF to images and storing to a folder

In [6]:
!apt-get -q update
!apt-get install -yq poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,944 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

In [7]:
from pdf2image import convert_from_path
from tqdm import tqdm
import shutil
import os

## 3.1 Delete the pdf_images folder if exists and then create a new folder

In [8]:
output_folder='/kaggle/working/page_images'
print(f"Cleaning output folder {output_folder}")
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

Cleaning output folder /kaggle/working/page_images


## 3.2 Convert all the PDf pages to images and store in pdf_images folder

In [9]:
pdf_folder='/kaggle/input/prusa-mks3-5-pdf-data'

pdf_files=[file for file in os.listdir(pdf_folder) if file.lower().endswith('.pdf')]
len(pdf_files)

299

In [10]:
image_paths=[]
for pdf_file in tqdm(pdf_files,desc='Processing PDF files'):
    pdf_path=os.path.join(pdf_folder,pdf_file)
    pdf_name=os.path.splitext(pdf_file)[0]
    pdf_output_folder=os.path.join(output_folder,pdf_name)
    os.makedirs(pdf_output_folder,exist_ok=True)
    
    
    try:
        images=convert_from_path(pdf_path)
        for i,image in enumerate(images):
            image_path=os.path.join(pdf_output_folder,f"page_{i+1}.png")
            image_paths.append(image_path)
            image.save(image_path,"PNG")
    except Exception as e:
        print(f"Failed to convert {pdf_file}:{e}")

Processing PDF files: 100%|██████████| 299/299 [02:43<00:00,  1.83it/s]


# 4. Create the embeddings for the PDF page images

In [11]:
!pip -q install spaces

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 33.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 109.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00


In [12]:
import spaces
from typing import List
from PIL import Image

In [13]:
@spaces.GPU
def get_images(folder_path: str) -> List[Image.Image]:
    images = []
    folders = os.listdir(folder_path)
    
    for folder in folders:
        subfolder_path = os.path.join(folder_path, folder)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                try:
                    img = Image.open(file_path)
                    images.append(img)
                except Exception as e:
                    print(f"Skipping {file_path}: {e}")
        else:
            try:
                img = Image.open(subfolder_path)
                images.append(img)
            except Exception as e:
                print(f"Skipping {subfolder_path}: {e}")
    
    return images

image_list=get_images('/kaggle/working/page_images')
len(image_list)

458

In [14]:
datalaoder=DataLoader(
    dataset=ListDataset[str](image_list),
    batch_size=10,
    shuffle=False,
    collate_fn=lambda x:processor.process_images(x),
)

In [15]:
ds:List[torch.Tensor]=[]
for batch_doc in tqdm(datalaoder):
    with torch.no_grad():
        batch_doc={k:v.to(model.device) for k,v in batch_doc.items()}
        embeddings_doc=model(**batch_doc)
    ds.extend(list(torch.unbind(embeddings_doc.to(device))))
ds_np=[d.float().cpu().numpy() for d in ds]


100%|██████████| 46/46 [08:43<00:00, 11.38s/it]


In [16]:
print("Demo embeddings: \n",ds_np[:1])
print("\nPrint the number of embeddings created: ",len(ds_np))

Demo embeddings: 
 [array([[-0.20019531,  0.12597656,  0.18261719, ...,  0.02526855,
        -0.00136566, -0.11523438],
       [-0.2265625 ,  0.05444336,  0.203125  , ...,  0.06982422,
         0.01470947, -0.02331543],
       [-0.02514648,  0.11669922,  0.11181641, ...,  0.03369141,
         0.02685547, -0.02160645],
       ...,
       [-0.01287842, -0.10595703,  0.13476562, ..., -0.05273438,
        -0.02819824,  0.03417969],
       [-0.27929688,  0.03540039,  0.07470703, ...,  0.01239014,
         0.00283813, -0.02783203],
       [-0.25390625,  0.05615234,  0.26757812, ...,  0.0703125 ,
        -0.10986328, -0.12402344]], dtype=float32)]

Print the number of embeddings created:  458


# 5. Initialising Melvus and Create Collection

In [17]:
!pip install -q pymilvus

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 84.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 39.4 MB/s eta 0:00:00:00:0100:01


In [ ]:
from pymilvus import MilvusClient, DataType
from dotenv import load_dotenv

load_dotenv()
db_token = os.getenv("VDB_TOKEN")
uri=os.getenv("URI")
collection_name='prusa_printer'
milvus_uri=uri
create_collection=True
vector_dims=128

client=MilvusClient(uri=milvus_uri,token=db_token)
if client.has_collection(collection_name=collection_name):
    #Load the exisiting collection
    client.load_collection(collection_name=collection_name)
    print("Using the exisiting collection.")
if create_collection:
    if client.has_collection(collection_name=collection_name):
        #I exisit delete the old schema
        client.drop_collection(collection_name=collection_name)
        print("Deleting the old collection...")
        
    #Create a schema   
    schema=client.create_schema(auto_id=True,enable_dynamic_fields=True)
    schema.add_field(field_name='pk',datatype=DataType.INT64,is_primary=True)
    schema.add_field(field_name='vector',datatype=DataType.FLOAT_VECTOR,dim=vector_dims)
    schema.add_field(field_name='seq_id',datatype=DataType.INT16)
    schema.add_field(field_name='doc_id',datatype=DataType.INT64)
    schema.add_field(field_name='doc',datatype=DataType.VARCHAR,max_length=65535)
    
    #Create the collection using the schema
    client.create_collection(collection_name=collection_name,
                             schema=schema)

    #Create index for vector search
    client.release_collection(collection_name=collection_name)
    client.drop_index(collection_name=collection_name,
                      index_name='vector')
    index_params=client.prepare_index_params()
    index_params.add_index(
        field_name='vector',
        index_name='vector_index',
        index_type="IVF_FLAT",
        metric_type="L2",
        params={
            "nlist":128
        },
    )
    client.create_index(collection_name=collection_name,
                        index_params=index_params,
                        sync=True)
    print("Collection is created annd indexed")

Collection is created annd indexed


# Insert the embeddings in the VectorDB

In [54]:
image_data=[]
for idx,(vec,path) in enumerate(zip(ds_np,image_paths)):
    image_data.append({
        "colbert_vecs":vec,
        "filepath":path, #!! Need to be the path for the image
        "doc_id":idx
    })
image_data[:3]

[{'colbert_vecs': array([[-0.20019531,  0.12597656,  0.18261719, ...,  0.02526855,
          -0.00136566, -0.11523438],
         [-0.2265625 ,  0.05444336,  0.203125  , ...,  0.06982422,
           0.01470947, -0.02331543],
         [-0.02514648,  0.11669922,  0.11181641, ...,  0.03369141,
           0.02685547, -0.02160645],
         ...,
         [-0.01287842, -0.10595703,  0.13476562, ..., -0.05273438,
          -0.02819824,  0.03417969],
         [-0.27929688,  0.03540039,  0.07470703, ...,  0.01239014,
           0.00283813, -0.02783203],
         [-0.25390625,  0.05615234,  0.26757812, ...,  0.0703125 ,
          -0.10986328, -0.12402344]], dtype=float32),
  'filepath': '/kaggle/working/page_images/27. Installing the Y-carriage-1/page_1.png',
  'doc_id': 0},
 {'colbert_vecs': array([[ 0.07080078,  0.03417969,  0.08251953, ...,  0.06079102,
           0.08984375,  0.11474609],
         [-0.07128906,  0.00738525, -0.01965332, ...,  0.06787109,
           0.04956055,  0.14355469],
 

In [58]:
eg1=image_data[0]
seq1=eg1['colbert_vecs'][0]
seq_last=eg1['colbert_vecs'][1030]
seq_last

array([-2.53906250e-01,  5.61523438e-02,  2.67578125e-01, -2.92968750e-02,
       -5.44433594e-02,  8.59375000e-02, -9.71679688e-02, -9.58251953e-03,
       -1.01318359e-02, -3.66210938e-02, -1.20605469e-01, -7.91015625e-02,
       -8.44726562e-02,  9.13085938e-02,  8.34960938e-02,  1.63574219e-02,
        4.73022461e-03,  2.89916992e-03,  1.31835938e-01, -5.15136719e-02,
       -3.01513672e-02,  3.32031250e-02, -7.47070312e-02, -1.41601562e-01,
       -5.85937500e-02,  4.60815430e-03,  7.91015625e-02,  1.19781494e-03,
       -1.18164062e-01, -5.51757812e-02, -1.61132812e-01,  1.16699219e-01,
       -4.59671021e-04, -2.36816406e-02, -3.54003906e-03,  4.88281250e-02,
        6.93359375e-02, -4.93164062e-02,  9.81445312e-02, -4.11987305e-03,
        8.69140625e-02, -9.13085938e-02,  6.68945312e-02,  1.80664062e-02,
       -4.12597656e-02, -1.63085938e-01,  3.19824219e-02, -5.10253906e-02,
       -9.03320312e-02, -3.63769531e-02,  7.23266602e-03, -1.06445312e-01,
        2.72216797e-02,  

In [56]:
for data in image_data:
    colbert_vecs=data['colbert_vecs']
    if isinstance(colbert_vecs, np.ndarray) and len(colbert_vecs.shape) == 2:
        colbert_vecs = colbert_vecs.tolist()
    seq_length=len(colbert_vecs)
    seq_ids=list(range(seq_length))
    docs_ids = [data["doc_id"]] * seq_length
    docs = [data["filepath"]] * seq_length

    records=[{
        "vector":colbert_vecs[i],
        "seq_id":seq_ids[i],
        "doc_id":docs_ids[i],
        "doc":docs[i]
    }for i in range(seq_length)]
    # for rec in records:
    #     print(f"seq_id: {rec.get('seq_id')}")
    #     print(f"doc_id: {rec.get('doc_id')}")
    #     print(f"doc: {rec.get('doc')}")
    #     print("-" * 40)
    client.insert(collection_name=collection_name,data=records)
print("All Data inserted successfully")

All Data inserted successfully


# Search for the query 

In [60]:
import concurrent.futures
@spaces.GPU
def search_and_rerank(query_vectors: np.ndarray, topk: int = 5):
    query_vectors = np.atleast_2d(query_vectors)
    search_params = {"metric_type": "L2", "params": {}}
    print("searching...")

    # Step 1: Milvus Search
    results = client.search(
        collection_name,
        data=query_vectors.tolist(),
        limit=50,
        output_fields=["vector", "seq_id", "doc_id"],
        search_params=search_params,
    )

    # Step 2: Collect document IDs
    doc_ids = {r["entity"]["doc_id"] for result in results for r in result}

    # Step 3: Reranking
    print("reranking...")

    def rerank_single_doc(doc_id):
        doc_colbert_vecs = client.query(
            collection_name=collection_name,
            filter=f"doc_id == {doc_id}",
            output_fields=["seq_id", "vector", "doc"],
            limit=1000,
        )
        if not doc_colbert_vecs:
            return (0.0, doc_id)
        doc_vecs = np.vstack([vec["vector"] for vec in doc_colbert_vecs])
        score = np.dot(query_vectors, doc_vecs.T).max(1).sum()
        return (score, doc_id)

    scores = []
    max_workers = min(8, len(doc_ids))
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(rerank_single_doc, doc_id): doc_id for doc_id in doc_ids}
        for future in concurrent.futures.as_completed(futures):
            scores.append(future.result())

    # Step 4: Sort and return top-K
    print("returning...")
    scores.sort(key=lambda x: x[0], reverse=True)
    return scores[:topk]

In [61]:
scores=search_and_rerank(seq1,topk=3)
scores

searching...
reranking...
returning...


[(0.9947926, 0), (0.9932466, 384), (0.99171203, 231)]